In [1]:
import os
import numpy as np
import scipy.io
import tensorflow as tf
import tensorflow_datasets as tfds
from tqdm import tqdm
import cv2
from sklearn.utils import gen_batches
import shutil
from correlation_analysis import CCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from models import build_deepCCA_model, compute_loss, CCA, compute_regularization, compute_termination_score

from tensorboard_utillities import write_scalar_summary, write_image_summary, write_PCC_summary, write_gradients_summary_mean, write_poly
from tensorboard_utillities import create_grid_writer

In [2]:
eeg = scipy.io.loadmat('eeg_data.mat')['data'][:, :, 0:172]
meg = scipy.io.loadmat('meg_data.mat')['data']
labels = [int(i) for i in scipy.io.loadmat('labels.mat')['L']]

In [3]:
def split_data(data, labels):
    channels, samples, trials = data.shape
    y = np.array([label*np.ones(161) for label in labels]).flatten()
    X = np.reshape(data, (channels, samples*trials)).T

    X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.10)
    X_test, X_val, y_test, y_val = train_test_split(X_tmp, y_tmp, test_size=0.50)

    return {'train': {'data': X_train.T, 'labels': y_train},
            'validation': {'data': X_val.T, 'labels': y_val},
            'test': {'data': X_test.T, 'labels': y_test}}

In [4]:
eeg_data = split_data(eeg, labels)
meg_data = split_data(meg, labels)
eeg_data['train']['labels'].shape, meg_data['test']['labels'].shape, meg_data['validation']['labels'].shape 

((24922,), (1385,), (1385,))

In [15]:
def batch_data(data, batch_size):
    channels, samples = data.shape
    num_batches = samples // batch_size

    tmp = np.zeros(shape=(num_batches, batch_size, channels), dtype=np.float32)
    for batch_idx, indeces in enumerate(gen_batches(samples, batch_size)):
        tmp[batch_idx] = data[:, indeces].T

    return tf.convert_to_tensor(tmp, dtype=tf.float32), num_batches


def train(train_data, train_labels, validation_data, validation_labels,
          max_epochs, batch_size, log_path, model_path, shared_dim=5, iter_idx=1):

    y_1, num_batches_1 = batch_data(train_data[0], batch_size)
    y_2, num_batches_2 = batch_data(train_data[1], batch_size)
    assert num_batches_1 == num_batches_2

    writer = create_grid_writer(root_dir=log_path, params=['deepCCA', f'v{iter_idx}', f'Shared dim {shared_dim}'])
    try:
        mpath = f'{model_path}/v{iter_idx}-SharedDim-{shared_dim}-BatchSize-{batch_size}'
        model = tf.keras.models.load_model(mpath)
        print('Loading existing model.\n')
    except:
        model = build_deepCCA_model([256, 256, 256], shared_dim)

    termination_condition = False
    termination_score_history = list()
    
    for epoch in tqdm(range(max_epochs), desc='Epochs'):
        if termination_condition:
            break
        
        losses, cca_losses, reg_losses = list(), list(), list()
        intermediate_outputs = list()
        for batch_idx in range(num_batches_1):
            batch_y1, batch_y2 = y_1[batch_idx], y_2[batch_idx]
            with tf.GradientTape() as tape:
                tape.watch([batch_y1, batch_y2])
                fy_1, fy_2 = model([batch_y1, batch_y2])
                cca_loss = compute_loss(fy_1, fy_2)
                reg_loss = compute_regularization(model, lambda_reg=1e-6)
                if epoch >= 5:
                    loss = cca_loss + reg_loss
                else:
                    loss = cca_loss
                gradients = tape.gradient(loss, model.trainable_variables)
                model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                intermediate_outputs.append((fy_1, fy_2))
                # Keeping track of losses
                cca_losses.append(cca_loss); reg_losses.append(reg_loss); losses.append(loss)
                
        if epoch % 25 == 0:
            tmp = list()
            for batch_idx in range(num_batches_2):
                batched_fy_1, batched_fy_2 = intermediate_outputs[batch_idx]
                B1, B2, epsilon, omega, ccor = CCA(batched_fy_1, batched_fy_2)
                tmp.append(ccor)
            avg_ccor = tf.math.reduce_mean(tmp, axis=0)
            static_part = [(tf.math.reduce_mean(losses), 'Loss/Total'),
                           (tf.math.reduce_mean(cca_losses), 'Loss/CCA'),
                           (tf.math.reduce_mean(reg_losses), 'Loss/Regularization'),
                           (tf.math.reduce_mean(termination_score_history[-5:]), 'Score/Value')]
            dynamic_part = [(cval, f'Canonical correlation/{idx})') for idx, cval in enumerate(avg_ccor)]
            write_scalar_summary(
                writer=writer,
                epoch=epoch,
                list_of_tuples=static_part + dynamic_part
            )
            
        if epoch % 50 == 0:
            val_fy_1, val_fy_2 = model([validation_data[0].T, validation_data[1].T])
            tscore_view1 = compute_termination_score(fy_1, train_labels[0], val_fy_1, validation_labels[0])
            tscore_view2 = compute_termination_score(fy_2, train_labels[1], val_fy_2, validation_labels[1])
            tscore = tf.reduce_mean(np.array([tscore_view1, tscore_view2]))
            termination_score_history.append(tscore)
            if (tscore.numpy() < np.mean(termination_score_history[-6:-1])) and (epoch >= 300):
                termination_condition = True
                
        if epoch % 250 == 0:
            try:
                os.makedirs(model_path)
            except FileExistsError:
                print('MODELS PATH exists, saving data.')
            finally:
                model.save(f'{model_path}/v{iter_idx}-SharedDim-{shared_dim}-BatchSize-{batch_size}/model.tf', overwrite=True)
                with open(f'{model_path}/v{iter_idx}-SharedDim-{shared_dim}-BatchSize-{batch_size}/modellog.txt', 'a+') as f:
                    f.write(f'Saving model at epoch {epoch}\n')
    
                        
def test(model, train_data, train_labels, validation_data, validation_labels, test_data, test_labels):
    p_fy_1, p_fy_2 = model([train_data[0].T, train_data[1].T])
    v_fy_1, v_fy_2 = model([validation_data[0].T, validation_data[1].T])
    
    s_fy_1, s_fy_2 = tf.concat([p_fy_1, v_fy_1], axis=0), tf.concat([p_fy_2, v_fy_2], axis=0)
    labels_1 = tf.concat([train_labels[0], validation_labels[0]], axis=0)
    labels_2 = tf.concat([train_labels[1], validation_labels[1]], axis=0)
    
    fy_1, fy_2 = model([test_data[0].T, test_data[1].T])
    score_view1 = compute_termination_score(s_fy_1, labels_1, fy_1, test_labels[0])
    score_view2 = compute_termination_score(s_fy_2, labels_2, fy_2, test_labels[1])
    
    return [score_view1, score_view2]

def test_raw(train_data, train_labels, validation_data, validation_labels, test_data, test_labels):
    view_1 = tf.concat([train_data[0].T, validation_data[0].T], axis=0)
    view_2 = tf.concat([train_data[1].T, validation_data[1].T], axis=0)
    
    labels_1 = tf.concat([train_labels[0], validation_labels[0]], axis=0)
    labels_2 = tf.concat([train_labels[1], validation_labels[1]], axis=0)
    
    score_view1 = compute_termination_score(view_1, labels_1, tf.convert_to_tensor(test_data[0].T), test_labels[0])
    score_view2 = compute_termination_score(view_2, labels_2, tf.convert_to_tensor(test_data[1].T), test_labels[1])
    
    return [score_view1, score_view2]

In [16]:
results = list()
raw_results = list()

for iter_idx in range(5):
    desc = 'MEEG-EEG'
    LOGPATH = f'{os.getcwd()}/LOG/{desc}'
    MODELSPATH = f'{os.getcwd()}/MODELS/{desc}'
    
    eeg_data = split_data(eeg, labels)
    meg_data = split_data(meg, labels)

    train(train_data=[eeg_data['train']['data'], meg_data['train']['data']], 
          train_labels=[eeg_data['train']['labels'], meg_data['train']['labels']],
          validation_data=[eeg_data['validation']['data'], meg_data['validation']['data']], 
          validation_labels=[eeg_data['validation']['labels'], meg_data['validation']['labels']],
          max_epochs=2000, batch_size=24922, log_path=LOGPATH, model_path=MODELSPATH, iter_idx=iter_idx)
    
    model = model = tf.keras.models.load_model(f'{MODELSPATH}/SharedDim-5-BatchSize-24922/model.tf')
    tmp = test(model,
               train_data=[eeg_data['train']['data'], meg_data['train']['data']],
               train_labels=[eeg_data['train']['labels'], meg_data['train']['labels']],
               validation_data=[eeg_data['validation']['data'], meg_data['validation']['data']], 
               validation_labels=[eeg_data['validation']['labels'], meg_data['validation']['labels']],
               test_data=[eeg_data['test']['data'], meg_data['test']['data']],
               test_labels=[eeg_data['test']['labels'], meg_data['test']['labels']])
    results.append(tmp)
    
    tmp = test_raw(train_data=[eeg_data['train']['data'], meg_data['train']['data']],
                   train_labels=[eeg_data['train']['labels'], meg_data['train']['labels']],
                   validation_data=[eeg_data['validation']['data'], meg_data['validation']['data']], 
                   validation_labels=[eeg_data['validation']['labels'], meg_data['validation']['labels']],
                   test_data=[eeg_data['test']['data'], meg_data['test']['data']],
                   test_labels=[eeg_data['test']['labels'], meg_data['test']['labels']])
    raw_results.append(tmp)
    
    

Model: "deepCCA"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Layer_1 (InputLayer)      [(None, 130)]        0                                            
__________________________________________________________________________________________________
Input_Layer_2 (InputLayer)      [(None, 151)]        0                                            
__________________________________________________________________________________________________
View_1_Layer_H1 (Dense)         (None, 256)          33536       Input_Layer_1[0][0]              
__________________________________________________________________________________________________
View_2_Layer_H1 (Dense)         (None, 256)          38912       Input_Layer_2[0][0]              
____________________________________________________________________________________________

Epochs:   0%|                                                                                                 | 0/2000 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


MODELS PATH exists, saving data.


INFO:tensorflow:Assets written to: /Users/alexander/Documents/Uni/Work/Repositories/deep-CCA-tf/MODELS/MEEG-EEG/v0-SharedDim-5-BatchSize-24922/model.tf/assets


INFO:tensorflow:Assets written to: /Users/alexander/Documents/Uni/Work/Repositories/deep-CCA-tf/MODELS/MEEG-EEG/v0-SharedDim-5-BatchSize-24922/model.tf/assets
Epochs:  12%|██████████▉                                                                            | 250/2000 [01:49<12:32,  2.33it/s]

...

INFO:tensorflow:Assets written to: /Users/alexander/Documents/Uni/Work/Repositories/deep-CCA-tf/MODELS/MEEG-EEG/v4-SharedDim-5-BatchSize-24922/model.tf/assets


INFO:tensorflow:Assets written to: /Users/alexander/Documents/Uni/Work/Repositories/deep-CCA-tf/MODELS/MEEG-EEG/v4-SharedDim-5-BatchSize-24922/model.tf/assets
Epochs:  12%|██████████▉                                                                            | 250/2000 [01:47<12:17,  2.37it/s]

MODELS PATH exists, saving data.


INFO:tensorflow:Assets written to: /Users/alexander/Documents/Uni/Work/Repositories/deep-CCA-tf/MODELS/MEEG-EEG/v4-SharedDim-5-BatchSize-24922/model.tf/assets


INFO:tensorflow:Assets written to: /Users/alexander/Documents/Uni/Work/Repositories/deep-CCA-tf/MODELS/MEEG-EEG/v4-SharedDim-5-BatchSize-24922/model.tf/assets
Epochs:  15%|█████████████                                                                          | 301/2000 [02:10<12:14,  2.31it/s]
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [17]:
1-np.mean(raw_results, axis=0), 1-np.mean(results, axis=0)

(array([0.49285199, 0.49718412]), array([0.48418773, 0.49891697]))

In [18]:
results, raw_results

([[0.527797833935018, 0.5010830324909747],
  [0.5371841155234657, 0.4815884476534296],
  [0.48592057761732854, 0.5054151624548736],
  [0.503971119133574, 0.5140794223826715],
  [0.524187725631769, 0.5032490974729242]],
 [[0.4844765342960289, 0.496028880866426],
  [0.5314079422382672, 0.5010830324909747],
  [0.5068592057761733, 0.49097472924187724],
  [0.503971119133574, 0.5104693140794224],
  [0.5090252707581228, 0.5155234657039711]])